In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

In [2]:
book_info_total = pd.read_pickle('./book_info_total.pkl')
book_info_total

,title,author,rating,category,tag,keyword,image,title_concat,feature
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L,매매의기술,박병창 주식 투자 재테크 재테크일반 경제 경영 증권 주식투자입문 주식투자
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L,부의시나리오,오건영 경제 경영 투자 재테크 재테크일반 투자트렌드 포트폴리오 삼프로TV 금융시...
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L,라오어의미국주식무한매수법,라오어 주식 투자 재테크 경제 경영 증권 미국주식
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L,부자아빠가난한아빠20주년특별기념판,로버트 기요사키 안진환 경제 경영 투자 재테크 재테크일반 돈모으고싶을때 금융문맹...
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L,나는당신이주식공부를시작했으면좋겠습니다,이효석 주식 투자 재테크 경제 경영 증권 삼프로TV 주식투자입문 주식투자 투자 ...
...,...,...,...,...,...,...,...,...,...
4239,뇌를 최적화시키는 생활명상 10분,신재한 임운나,NaN,뇌과학 뇌과학 일반 자연과학,,마음 상태 의식 목적 명상,http://image.yes24.com/goods/90584042/L,뇌를최적화시키는생활명상10분,신재한 임운나 뇌과학 뇌과학 일반 자연과학 마음 상태 의식 목적 명상
4240,"아빠, 천체관측 떠나요!",조상호,9.7,천문학 자연과학 별자리와 천체관측,,,http://image.yes24.com/goods/90175509/L,"아빠,천체관측떠나요!",조상호 천문학 자연과학 별자리와 천체관측
4241,인체 의학 도감 베스트 4종 세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지,NaN,의학 재미있는 인체이야기 자연과학 인체 건강에세이 건강기타 건강 취미 약학,,,http://image.yes24.com/goods/89970464/L,인체의학도감베스트4종세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지 ...
4242,세상에서 가장 재미있는 생물학,데이브 웨스너 래리 고닉 김소정,10.0,생물학 생명과학 재미있는 생명이야기 자연과학,,생물학 가장 웨스 세상 시리 하버드 만화 데이브 과학 만점,http://image.yes24.com/goods/89907468/L,세상에서가장재미있는생물학,데이브 웨스너 래리 고닉 김소정 생물학 생명과학 재미있는 생명이야기 자연과학 생...


In [3]:
# 사용자가 검색어로 넣는 책 이름의 띄어쓰기가 정확하지 않아도 검색 가능하도록 함
def get_book_idx_by_title(book_title):
    # 책 제목이 검색어와 완전히 일치하지 않아도 포함만 되면 검색 되도록 함
    book_title_concat = book_title.replace(' ', '')
    book_idx = book_info_total.index[(book_info_total['title_concat'] == book_title_concat)].to_list()
    if book_idx == []:
        book_idx = book_info_total.index[(book_info_total['title_concat'].str.contains(book_title_concat))].to_list()
    if book_idx:
        i = random.randrange(len(book_idx))
        return book_idx[i]
    else:
        return None

def get_book_title(book_idx):
    if book_idx in book_info_total.index.values:
        return book_info_total.loc[book_idx]['title']
    else:
        return None
    
# 사용자가 검색어로 넣은 단어가 책의 feature keywords 중 하나와 매치되는지
def get_book_idx_by_keyword(keyword):
    book_idx = book_info_total.index[(book_info_total['feature'].str.contains(keyword))].to_list()
    if book_idx:
        i = random.randrange(len(book_idx))
        return book_idx[i]
    else:
        return None

In [4]:
# min_df를 1로 설정해줌으로써 한번이라도 노출된 정보도 다 고려함
# ngram_range: n_gram 범위 지정 연속으로 나오는 단어들의 순서도 고려함
tf = TfidfVectorizer(min_df=1, ngram_range=(1,5))
tfidf_matrix = tf.fit_transform(book_info_total['feature'])
tfidf_matrix

<4244x165244 sparse matrix of type '<class 'numpy.float64'>'
	with 286929 stored elements in Compressed Sparse Row format>

In [5]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.07425905, 0.14541622, ..., 0.        , 0.        ,
        0.        ],
       [0.07425905, 1.        , 0.04862108, ..., 0.        , 0.        ,
        0.        ],
       [0.14541622, 0.04862108, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.00415701,
        0.00476445],
       [0.        , 0.        , 0.        , ..., 0.00415701, 1.        ,
        0.00457294],
       [0.        , 0.        , 0.        , ..., 0.00476445, 0.00457294,
        1.        ]])

In [45]:
# 제목으로 검색
def similar_books_by_title(title, n=10):
    title = title.replace(' ', '')
    # 해당 제목의 책의 index를 구한다.
    book_idx = get_book_idx_by_title(title)
    if book_idx is None:
        #print("해당 도서는 데이터셋에 존재하지 않습니다. 다른 도서나 키워드로 다시 검색해주세요.")
        return None
    else:
        # 모든 책에 대해서 해당 책과의 cosine similarity를 구한다.
        book_similarities = list(enumerate(cosine_sim[book_idx]))
        # cosine similarity에 따라 책들을 정렬한다.
        book_similarities = sorted(book_similarities, key=lambda x:x[1], reverse=True)
        most_similar_books = list(map(lambda x: (get_book_title(x[0]), x[0]), book_similarities[:n+1]))       
        most_similar_books = list(filter(lambda x: title != x[0].replace(' ',''), most_similar_books))
        book_indices = []
        # 검색어가 책 제목에 포함되는 경우, 첫번째 검색 결과부터 n개
        if len(most_similar_books) > n:
            for i, j in book_similarities[:n]:
                book_indices.append(i)
        # 검색어가 책 제목과 완전히 일치하는 경우, 두번째 검색 결과부터 n개 (첫번째 검색 결과는 자기자신이므로)
        else:
            for i, j in book_similarities[1:n+1]:
                book_indices.append(i) 
        return(book_indices)
        
# 카테고리, 저자, 키워드로 검색
def similar_books_by_keyword(keyword, n=10):
    # 해당 제목의 책의 index를 구한다.
    book_idx = get_book_idx_by_keyword(keyword)
    if book_idx is None:
        #print("해당 키워드는 검색이 불가능합니다. 다른 도서나 키워드, 카테고리, 저자명으로 다시 검색해주세요.\n혹시 여러 개의 키워드를 입력하였다면 개수를 줄여주세요.")
        return None
    else:
        # 모든 책에 대해서 해당 책과의 cosine similarity를 구한다.
        book_similarities = list(enumerate(cosine_sim[book_idx]))
        # cosine similarity에 따라 책들을 정렬한다.
        book_similarities = sorted(book_similarities, key=lambda x:x[1], reverse=True)
        most_similar_books = list(map(lambda x: (get_book_title(x[0]), x[0]), book_similarities[:n]))       
        most_similar_books = list(filter(lambda x: keyword != x[0], most_similar_books))
        book_indices = []
        for i, j in book_similarities[:n]:
            book_indices.append(i)
        return(book_indices)

In [7]:
# 검색 결과 dataframe을 json으로 변환
def dataframe_to_json(dataframe):
    return dataframe.to_json(orient = 'records', force_ascii=False)

## /title

In [17]:
idx_1 = similar_books_by_title('단테의신곡')
result_df = book_info_total.loc[idx_1,['title', 'author', 'rating', 'category', 'tag', 'keyword', 'image']]
result_df

,title,author,rating,category,tag,keyword,image
1217,신곡 세트,단테 박상진,8.8,서양 고전문학 고전문학,이가격실화 요즘책방 책읽어드립니다,,http://image.yes24.com/goods/9387098/L
1135,데미안,헤르만 헤세 전영애,8.8,성장소설 노벨문학상 수상작가 해외문학상 수상작 독일소설 가족소설 테마소설,20세기고전 방탄책방 방탄RM이추천한책 북클러버의선택 안사면0원사면영원 요즘책방...,싱클,http://image.yes24.com/goods/176787/L
1450,원전번역 일리아스 오뒷세이아 세트,호메로스,9.1,서양 고전문학 고전문학,불멸의고전 알쓸신잡,전쟁 트로 유럽,http://image.yes24.com/goods/3516734/L
1505,셰익스피어 4대 비극 컬렉션 세트,윌리엄 셰익스피어,9.4,고전문학 시나리오 시 서양 고전문학 희곡,요즘책방 책읽어드립니다,가장 비극,http://image.yes24.com/goods/7979861/L
1400,걸리버여행기,조나단 스위프트 이종인,9.6,영미소설 영미 장편소설 고전문학 서양 고전문학,요즘책방 책읽어드립니다,,http://image.yes24.com/goods/78241925/L
1274,사람은 무엇으로 사는가,레프 톨스토이 홍대화,9.8,러시아소설 서양 고전문학 고전문학,,인생 변화 톨스 깨달,http://image.yes24.com/goods/97177454/L
1552,구토,장 폴 사르트르 임호경,9.2,프랑스소설 서양 고전문학 고전문학,20세기고전,사르트르 구토 철학 세기 의미 로캉,http://image.yes24.com/goods/96651059/L
1356,햄릿,윌리엄 셰익스피어 최종철,8.6,고전문학 시나리오 시 서양 고전문학 희곡,비극 불멸의고전 요즘책방 책읽어드립니다,,http://image.yes24.com/goods/57197/L
1467,홍계월전,유광수 홍선주,9.0,한국 고전문학 고전문학,,여성,http://image.yes24.com/goods/5840640/L
1290,이반 데니소비치 수용소의 하루,알렉산드르 이자에비치 솔제니친,9.0,러시아소설 서양 고전문학 고전문학,장소적제약과인간 참혹한수용소 20세기고전,작가 솔제 인간 사회 작품,http://image.yes24.com/goods/17377/L


In [51]:
result_json = dataframe_to_json(result_df)
print(result_json)

[{"title":"초판본 셰익스피어 4대 비극","author":"윌리엄 셰익스피어 김민애 한우리 ","rating":9.9,"category":" 고전문학 시나리오 시 서양 고전문학 희곡","tag":"","keyword":"","image":"http:\/\/image.yes24.com\/goods\/89726621\/L"},{"title":"셰익스피어 5대 희극","author":"윌리엄 셰익스피어 셰익스피어연구회 ","rating":10.0,"category":" 고전문학 시나리오 시 서양 고전문학 희곡","tag":"","keyword":"사랑 셰익 ","image":"http:\/\/image.yes24.com\/goods\/92099654\/L"},{"title":"셰익스피어 4대 비극 컬렉션 세트","author":"윌리엄 셰익스피어 ","rating":9.4,"category":" 고전문학 시나리오 시 서양 고전문학 희곡","tag":" 요즘책방 책읽어드립니다","keyword":"가장 비극 ","image":"http:\/\/image.yes24.com\/goods\/7979861\/L"},{"title":"햄릿","author":"윌리엄 셰익스피어 최종철 ","rating":8.6,"category":" 고전문학 시나리오 시 서양 고전문학 희곡","tag":" 비극 불멸의고전 요즘책방 책읽어드립니다","keyword":"","image":"http:\/\/image.yes24.com\/goods\/57197\/L"},{"title":"파우스트 1","author":"요한 볼프강 폰 괴테 ","rating":8.7,"category":" 고전문학 시나리오 시 서양 고전문학 희곡","tag":"","keyword":"괴테 ","image":"http:\/\/image.yes24.com\/goods\/2548\/L"},{"title":"파우스트 2","author":"요한 볼프강 폰 괴테 ","rating":8.6,"category":" 고전문

In [21]:
idx_2 = similar_books_by_title('리스본행')
result_df = book_info_total.loc[idx_2,['title', 'author', 'rating', 'category', 'tag', 'keyword', 'image']]
result_df

해당 도서는 데이터셋에 존재하지 않습니다. 다른 도서나 키워드로 다시 검색해주세요.


KeyError: None

In [53]:
result_json = dataframe_to_json(result_df)
print(result_json)

[{"title":"적정한 삶 ","author":"김경일 ","rating":9.7,"category":"쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리","tag":"","keyword":"","image":"http:\/\/image.yes24.com\/goods\/98832633\/L"},{"title":"행복의 정복","author":"버트런드 러셀 이순희 ","rating":8.7,"category":"쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리","tag":"","keyword":"","image":"http:\/\/image.yes24.com\/goods\/1447038\/L"},{"title":"솔직하게, 상처 주지 않게","author":"전미경 ","rating":9.6,"category":"쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리","tag":" 책읽아웃 마음다스리기 감정컨트롤","keyword":"","image":"http:\/\/image.yes24.com\/goods\/93620595\/L"},{"title":"가족의 두 얼굴","author":"최광현 ","rating":9.9,"category":"쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리","tag":"","keyword":"가족 상처 아픔 ","image":"http:\/\/image.yes24.com\/goods\/99985418\/L"},{"title":"그림으로 읽는 생생 심리학","author":"이소라 남종호 ","rating":null,"category":"쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리","tag":"","keyword":"심리 생생 그림 ","image":"http:\/\/image.yes24.com\/goods\/95713544\/L"},{"title":"사람이 힘겨운 당신을 위한 관계의 심리학","author":"최광현 ","rating":9.8,"category":"쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리","tag"

## /keyword

In [40]:
idx_3 = similar_books_by_keyword('책읽어드립니다')
result_df = book_info_total.loc[idx_3,['title', 'author', 'rating', 'category', 'tag', 'keyword', 'image']]
result_df

,title,author,rating,category,tag,keyword,image
3771,이기적 유전자 The Selfish Gene,리처드 도킨스 홍영남 이상임,9.3,경향신문 유전자 동아일보 과학 바이러스 재미있는 과학이야기 자연과학 미디어 추천 생...,내말이사이언스 요즘책방 책읽어드립니다 빨간책방 교양쌓기 알쓸신잡,,http://image.yes24.com/goods/65067259/L
4014,진화의 오리진,존 그리빈 메리 그리빈 권루시안,9.4,유전자 동아일보 과학 바이러스 재미있는 과학이야기 자연과학 미디어 추천 생명과학 과...,과학 물화생지,,http://image.yes24.com/goods/97812923/L
4116,이기적 감정,랜돌프 M. 네스 안진이 최재천,9.4,유전자 동아일보 한겨레 바이러스 자연과학 조선일보 미디어 추천 생명과학 재미있는 생...,정신분석 뇌과학,감정 진화 불안,http://image.yes24.com/goods/91607565/L
4011,내가 누구인지 뉴턴에게 물었다,김범준,9.8,경향신문 과학 재미있는 과학이야기 자연과학 미디어 추천,,,http://image.yes24.com/goods/98813545/L
4191,생명을 보는 마음,김성호,10.0,한겨레 자연과학 미디어 추천 생명과학 재미있는 생명이야기 생명과학 일반,,생명 자연 마음 대한 관찰 기록,http://image.yes24.com/goods/94492027/L
3839,공학의 눈으로 미래를 설계하라,연세대학교 공과대학,9.2,동아일보 과학 재미있는 과학이야기 자연과학 미디어 추천 과학 일반,,,http://image.yes24.com/goods/70826108/L
3796,종의 기원,찰스 로버트 다윈 장대익 최재천 강호정 김성한,9.0,경향신문 유전자 동아일보 과학 바이러스 자연과학 조선일보 과학자 미디어 추천 생명과...,과학 물화생지,,http://image.yes24.com/goods/76185005/L
3813,"질병 정복의 꿈, 바이오 사이언스",이성규,9.0,유전자 바이러스 자연과학 생명과학 재미있는 생명이야기 진화,,인류 건강 욕망 과학 시대,http://image.yes24.com/goods/69317616/L
3800,숲은 고요하지 않다,마들렌 치게 배명자 최재천,9.5,경향신문 동아일보 한겨레 중앙일보 자연과학 조선일보 미디어 추천 생명과학 재미있는 ...,바이오커뮤니케이션,,http://image.yes24.com/goods/99423761/L
4091,마블이 설계한 사소하고 위대한 과학,세바스찬 알바라도 박지웅,8.8,경향신문 과학 재미있는 과학이야기 자연과학 미디어 추천,,과학 마블 사랑,http://image.yes24.com/goods/81515954/L


In [39]:
result_json = dataframe_to_json(result_df)
print(result_json)

[{"title":"한중록","author":"혜경궁 홍씨 신동운 ","rating":9.4,"category":" 한국사 역사 고전문학 한국 고전문학 조선시대 한국문화","tag":" 요즘책방 책읽어드립니다 사도세자와영조 ","keyword":"","image":"http:\/\/image.yes24.com\/goods\/89604744\/L"},{"title":"초판본 한중록","author":"혜경궁 홍씨 박병성 ","rating":null,"category":" 한국사 역사 고전문학 한국 고전문학 조선시대 한국문화","tag":"","keyword":"","image":"http:\/\/image.yes24.com\/goods\/90372536\/L"},{"title":"한 권으로 읽는 쇄미록","author":"오희문 신병주 ","rating":9.8,"category":" 동아일보 한국사 역사 고전문학 한겨레 한국 고전문학 미디어 추천 조선시대 한국문화","tag":"","keyword":"쇄미 양반 희문 기록 ","image":"http:\/\/image.yes24.com\/goods\/95162646\/L"},{"title":"홍계월전","author":"유광수 홍선주 ","rating":9.0,"category":" 한국 고전문학 고전문학","tag":"","keyword":"여성 ","image":"http:\/\/image.yes24.com\/goods\/5840640\/L"},{"title":"신곡 세트","author":"단테 박상진 ","rating":8.8,"category":" 서양 고전문학 고전문학","tag":" 이가격실화  요즘책방 책읽어드립니다","keyword":"","image":"http:\/\/image.yes24.com\/goods\/9387098\/L"},{"title":"걸리버여행기","author":"조나단 스위프트 이종인 ","rating":9.6,"category":" 영미소설 영미 장편소설 고전문학 서양 

In [51]:
idx_4 = similar_books_by_keyword('알랭 드 보통')
result_df = book_info_total.loc[idx_4,['title', 'author', 'rating', 'category', 'tag', 'keyword', 'image']]
result_df

,title,author,rating,category,tag,keyword,image
1319,왜 나는 너를 사랑하는가,알랭 드 보통 정영목,8.5,프랑스소설 연애 사랑소설 테마소설,사랑의형태 봄날엔로맨스 하트시그널 사랑의의미 아이돌셀러,사랑 서로 사람 철학,http://image.yes24.com/goods/2659394/L
1716,불안,알랭 드 보통 정영목,8.8,외국 에세이 에세이 삶의 자세와 지혜,북클러버의선택 이정도는읽어야교양인 깊은에세이 불안,불안 보통 알랭 우리 지위,http://image.yes24.com/goods/6111280/L
1390,폭풍의 언덕,에밀리 브론테,8.8,영미소설 영미 장편소설 연애 사랑소설 테마소설,책읽아웃,소설,http://image.yes24.com/goods/1470060/L
1424,제인 에어 2,샬롯 브론테,9.2,영미소설 영미 장편소설 연애 사랑소설 테마소설,여성서사,에어 제인,http://image.yes24.com/goods/1426190/L
1968,여행의 기술,알랭 드 보통 정영목,8.5,여행 에세이 외국 에세이 여행에세이 여행 에세이,일상을위로하다,우리 여행 모든 국적 대한,http://image.yes24.com/goods/6042044/L
1405,제인 에어 1,샬롯 브론테,8.5,영미소설 영미 장편소설 연애 사랑소설 테마소설,여성서사 영화드라마원작,에어 제인,http://image.yes24.com/goods/1426188/L
1235,위대한 개츠비,스콧 피츠제럴드 김욱동,8.5,영미소설 영미 장편소설 영화와 드라마 원작 연애 사랑소설 테마소설,20세기고전 빨간책방 책끝을접다 영화드라마원작,개츠 미국,http://image.yes24.com/goods/370331/L
1181,자기 앞의 생,로맹 가리 용경식,9.2,공쿠르상 수상작 프랑스소설 연애 사랑소설 해외문학상 수상작 테마소설,KBS2TV북유럽 방탄RM이추천한책 나를일으키는문장 tvN비밀독서단추천도서 멜로...,모모 작가 공쿠르 사람 세상 무게 로맹 사랑 소설 자기,http://image.yes24.com/goods/359901/L
1396,낭만적 연애와 그 후의 일상,알랭 드 보통,8.8,동아일보 경향신문 YES24 올해의 책 중앙일보 2016년 올해의 책 후보도서 프...,봄날엔로맨스 책끝을접다,,http://image.yes24.com/goods/30547371/L
2710,알랭 드 보통의 영혼의 미술관,존 암스트롱 알랭 드 보통 김한영,8.6,교양 미술일반 예술 미술 예술사 예술일반,내손안의미술관 크기가큰책 이정도는읽어야교양인 교양쌓기,미술관 보통 영혼 보급 예술 알랭,http://image.yes24.com/goods/62758806/L


In [26]:
result_json = dataframe_to_json(result_df)
print(result_json)

[{"title":"고양이를 버리다","author":"무라카미 하루키 가오 옌 김난주 ","rating":8.8,"category":"경향신문 동아일보 에세이 한겨레 외국 에세이 조선일보 휴먼 에세이 미디어 추천","tag":" 아버지와나","keyword":"하루키 무라카미 아버지 대해 방울 작가 로서 이야 처음 고양이 ","image":"http:\/\/image.yes24.com\/goods\/93961269\/L"},{"title":"미친 세상을 이해하는 척하는 방법","author":"움베르토 에코 박종대 ","rating":9.2,"category":"경향신문 동아일보 에세이 한겨레 외국 에세이 중앙일보 조선일보 미디어 추천","tag":" 유작","keyword":"","image":"http:\/\/image.yes24.com\/goods\/97124427\/L"},{"title":"개와 함께한 10만 시간","author":"엘리자베스 마셜 토머스 정영문 ","rating":9.4,"category":"동아일보 에세이 한겨레 외국 에세이 조선일보 미디어 추천 동물 에세이","tag":"","keyword":"미샤 ","image":"http:\/\/image.yes24.com\/goods\/101511086\/L"},{"title":"바바리안 데이즈","author":"윌리엄 피네건 박현주 ","rating":9.0,"category":"경향신문 동아일보 에세이 한겨레 외국 에세이 조선일보 해외문학상 수상작 미디어 추천 퓰리처상 수상작","tag":" 여름맛 여름스포츠 빠져드는흡인력","keyword":"세계 파도 ","image":"http:\/\/image.yes24.com\/goods\/62280434\/L"},{"title":"힐빌리의 노래","author":"J. D. 밴스 김보람 ","rating":8.8,"category":"경향신문 동아일보 에세이 한겨레 외국 에세이 중앙일보 조선일보 미디어 추천","tag":" 빌게이츠추천 넷

# 검색 결과가 없는 경우

In [47]:
idx_5 = similar_books_by_title('바다사이등대')

if idx_5 == None:
    result = {"message": "해당 도서는 데이터셋에 존재하지 않습니다. 다른 도서나 키워드로 다시 검색해주세요."}
    print(result) # 여기에서 return jsonify(result) 해버리기

{'message': '해당 도서는 데이터셋에 존재하지 않습니다. 다른 도서나 키워드로 다시 검색해주세요.'}


In [48]:
idx_6 = similar_books_by_keyword('사랑 에세이 감성')

if idx_6 == None:
    result = {"message": "해당 키워드는 검색이 불가능합니다. 다른 도서나 키워드, 카테고리, 저자명으로 다시 검색해주세요.\n여러 개의 키워드를 입력하였다면 개수를 줄여주세요."}
    print(result) # 여기에서 return jsonify(result) 해버리기

{'message': '해당 키워드는 검색이 불가능합니다. 다른 도서나 키워드, 카테고리, 저자명으로 다시 검색해주세요.\n여러 개의 키워드를 입력하였다면 개수를 줄여주세요.'}
